In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scanpy as sc
import squidpy as sq
import gpflow
import tensorflow as tf
from GPcounts.GPcounts_Module import Fit_GPcounts
import statsmodels.formula.api as smf 
import statsmodels.api as sm

2023-03-14 19:57:19.976251: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-14 19:57:20.211039: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64:/cm/local/apps/gcc/9.2.0/lib:/cm/local/apps/gcc/9.2.0/lib64
2023-03-14 19:57:20.211068: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-14 19:57:20.993715: W tensorflow/compiler/xla/stream_executor/platform/default/d

In [2]:
adata = sc.read_h5ad(f"../../data/simulation/simulated/svgs_rep_0.h5ad")

In [3]:
Y = pd.DataFrame(data=adata.layers['counts'].todense(), 
                      index=adata.obs_names, 
                      columns=adata.var_names)

In [4]:
spatial_locations = pd.DataFrame(data=adata.obsm['spatial'], 
                                 index=adata.obs_names, columns=['x', 'y'])
spatial_locations['total_counts'] = adata.layers['counts'].sum(axis=1)

In [5]:
scales = []
for i in range(0, len(Y.columns)):
    model=smf.glm(formula = "Y.iloc[:, i]~0+spatial_locations['total_counts']", 
                  data=Y, 
                  family=sm.families.NegativeBinomial(sm.families.links.log())).fit()
    res = model.params[0]*spatial_locations['total_counts']
    scales.append(res)
    
scalesdf=pd.DataFrame(scales)
scalesdf=scalesdf.T

In [6]:
Y = Y.T

In [7]:
X = spatial_locations[['x', 'y']]
gp_counts = Fit_GPcounts(X, Y, scale=scalesdf, safe_mode=False)

In [8]:
log_likelihood_ratio = gp_counts.One_sample_test("Negative_binomial")

  0%|          | 0/250 [00:00<?, ?it/s]2023-03-14 19:57:25.700051: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-03-14 19:57:25.700086: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (hpc-head): /proc/driver/nvidia/version does not exist
2023-03-14 19:57:25.700692: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


  0%|          | 0/250 [00:06<?, ?it/s]
/home/rs619065/miniconda3/envs/svg/lib/python3.9/site-packages/GPcounts/GPcounts_Module.py:510: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self.sparse and self.lik_name is not "Gaussian":
/home/rs619065/miniconda3/envs/svg/lib/python3.9/site-packages/GPcounts/GPcounts_Module.py:522: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  and self.lik_name is not "Gaussian"
/home/rs619065/miniconda3/envs/svg/lib/python3.9/site-packages/GPcounts/GPcounts_Module.py:510: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self.sparse and self.lik_name is not "Gaussian":
/home/rs619065/miniconda3/envs/svg/lib/python3.9/site-packages/GPcounts/GPcounts_Module.py:522: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  and self.lik_name is not "Gaussian"
/home/rs619065/miniconda3/envs/svg/lib/python3.9/site-packages/GPcounts/GPcounts_Module.py:510: SyntaxWarning: "is not" with a literal. Did you mean "!="?
 

TypeError: in user code:

    File "/home/rs619065/miniconda3/envs/svg/lib/python3.9/site-packages/gpflow/optimizers/scipy.py", line 120, in _tf_eval  *
        loss, grads = _compute_loss_and_gradients(
    File "/home/rs619065/miniconda3/envs/svg/lib/python3.9/site-packages/gpflow/optimizers/scipy.py", line 223, in _compute_loss_and_gradients  *
        loss = loss_closure()
    File "/home/rs619065/miniconda3/envs/svg/lib/python3.9/site-packages/check_shapes/integration/tf.py", line 57, in wrapped_method  *
        return wrapped_function(self, *args, **kwargs)
    File "/home/rs619065/miniconda3/envs/svg/lib/python3.9/site-packages/check_shapes/decorator.py", line 120, in wrapped_function  *
        return func(*args, **kwargs)
    File "/home/rs619065/miniconda3/envs/svg/lib/python3.9/site-packages/gpflow/models/training_mixins.py", line 64, in training_loss  *
        return self._training_loss()  # type: ignore[attr-defined]
    File "/home/rs619065/miniconda3/envs/svg/lib/pytho

In [ ]:
results = gp_counts.calculate_FDR(log_likelihood_ratio)
results